# ДЗ №1

In [37]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 500)

In [38]:
data = pd.read_csv('smsspamcollection/SMSSpamCollection', delimiter="\t", names = ('type', 'text'))
data['is_spam'] = [0 if d_type == 'ham' else 1 for d_type in data.type]
data.head()

,type,text,is_spam
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives around here though",0


In [39]:
data.type.value_counts()

ham     4825
spam     747
Name: type, dtype: int64

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()#ngram_range=(1,2))
X = vectorizer.fit_transform(data.text)
print X.shape

(5572, 8713)


In [47]:
# какие получились признаки
print len(vectorizer.get_feature_names())
vectorizer.get_feature_names()[-5:]

104967


[u'\xfa1',
 u'\xfa1 20',
 u'\xfa1 20 poboxox36504w45wq',
 u'\u3028ud',
 u'\u3028ud evening']

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
    cls = LogisticRegression(C=c)
    res = cross_val_score(cls, X, data.is_spam, scoring="f1", cv=10)
    print "c = {:<7}   res = {:.2f}   std = {:.2f}".format(c, np.mean(res), np.std(res))

c = 0.01      res = 0.78   std = 0.05
c = 0.1       res = 0.90   std = 0.02
c = 0.5       res = 0.93   std = 0.02
c = 1         res = 0.93   std = 0.02
c = 5         res = 0.94   std = 0.02
c = 10        res = 0.94   std = 0.02
c = 100       res = 0.94   std = 0.02
c = 200       res = 0.94   std = 0.02
c = 500       res = 0.94   std = 0.02
c = 1000      res = 0.94   std = 0.02
c = 10000     res = 0.94   std = 0.02
c = 15000     res = 0.94   std = 0.02
c = 20000     res = 0.94   std = 0.02
c = 100000    res = 0.94   std = 0.02


Получили качество 0.94

In [43]:
cls.fit(X, data.is_spam)

LogisticRegression(C=100000, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [44]:
tests = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use fromyour phone now! Subscribe6GB",
        "FreeMsg: Txt: claim your reward of 3 hours talk time",
        "Have you visited the last lecture on physics?",
        "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
        "Only 99$"]

X_test = vectorizer.transform(tests)
cls.predict(X_test)

array([1, 1, 0, 0, 0])

In [45]:
#перебираем ngram
for ngram_range_ in [(2,2), (3,3), (1,3)]:
    vectorizer = CountVectorizer(ngram_range=ngram_range_)
    X = vectorizer.fit_transform(data.text)
    cls = LogisticRegression()
    res = cross_val_score(cls, X, data.is_spam, scoring="f1", cv=10)
    print "ngram_range = {}  res = {:.2f}".format(ngram_range_, np.mean(res))

ngram_range = (2, 2)  res = 0.82
ngram_range = (3, 3)  res = 0.73
ngram_range = (1, 3)  res = 0.93


In [46]:
#теперь для Naive Bayes
from sklearn.naive_bayes import MultinomialNB

for ngram_range_ in [(1,1), (2,2), (3,3), (1,3)]:
    vectorizer = CountVectorizer(ngram_range=ngram_range_)
    X = vectorizer.fit_transform(data.text)
    cls = MultinomialNB()
    res = cross_val_score(cls, X, data.is_spam, scoring="f1", cv=10)
    print "ngram_range = {}  res = {:.2f}".format(ngram_range_, np.mean(res))

ngram_range = (1, 1)  res = 0.93
ngram_range = (2, 2)  res = 0.65
ngram_range = (3, 3)  res = 0.38
ngram_range = (1, 3)  res = 0.89


Сравним tf*idf и CountVectorizer

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(data.text)

for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
    cls = LogisticRegression(C=c)
    res_TV = cross_val_score(cls, X_tfidf, data.is_spam, scoring="f1", cv=10)
    res_CV = cross_val_score(cls, X, data.is_spam, scoring="f1", cv=10)
    print "c = {:<7}   res_TV = {:.4f}   res_CV = {:.4f} ".format(c, np.mean(res_TV), np.mean(res_CV))

c = 0.01      res_TV = 0.0000   res_CV = 0.8101 
c = 0.1       res_TV = 0.0393   res_CV = 0.9025 
c = 0.5       res_TV = 0.7611   res_CV = 0.9244 
c = 1         res_TV = 0.8529   res_CV = 0.9251 
c = 5         res_TV = 0.9174   res_CV = 0.9252 
c = 10        res_TV = 0.9224   res_CV = 0.9260 
c = 100       res_TV = 0.9314   res_CV = 0.9283 
c = 200       res_TV = 0.9329   res_CV = 0.9283 
c = 500       res_TV = 0.9344   res_CV = 0.9283 
c = 1000      res_TV = 0.9344   res_CV = 0.9291 
c = 10000     res_TV = 0.9382   res_CV = 0.9321 
c = 15000     res_TV = 0.9382   res_CV = 0.9321 
c = 20000     res_TV = 0.9375   res_CV = 0.9321 
c = 100000    res_TV = 0.9375   res_CV = 0.9329 


Вывод следующий. Благодаря методу BagOfWords можно с помощью лишь логистической регрессии получить неплохую классификацию смс сообощений.